In [1]:
import pandas as pd
import numpy as np
import re

# recurrence relation
$$p_n = p_0 + nv_0 + \sum_{i=1}^{n}ia_0$$

$$p_n = p_0 + nv_0 +\frac{n(n+1)}{2}a_0$$

In [2]:
test_data='''p=< 3,0,0>, v=< 2,0,0>, a=<-1,0,0>
p=< 4,0,0>, v=< 0,0,0>, a=<-2,0,0>'''.splitlines()
real_data = open('input.txt').readlines()

In [3]:
parser = re.compile(r'(.=<([^,]+),([^,]+),([^,])+>(, )?)(.=<([^,]+),([^,]+),([^,])+>(, )?)(.=<([^,]+),([^,]+),([^,])+>(, )?)')

In [4]:
def parse_lines(lines):
    p = []
    v = []
    a = []
    for line in lines:
        l = parser.match(line)
        if l:
            p.append(list(map(int,l.group(2, 3, 4))))
            v.append(list(map(int,l.group(7, 8, 9))))
            a.append(list(map(int,l.group(12, 13, 14))))
    return map(lambda x: np.array(x, dtype=np.int64), [p, v, a]) # , dtype=np.float64

In [5]:
def manhattan(v):
    return abs(v).sum(axis=1)

def norm2(v):
    return (v**2).sum(axis=1)

In [6]:
# p, v, a = parse_lines(real_data)
p0, v0, a0 = parse_lines(real_data)
# n = 100000
# for i in range(n):
#     v = v + a
#     p = p + v

In [7]:
# def p_i(n):
#     return p0 + n*v0 + (n*(n+1)*a)/2

# np.where(manhattan(p_i(n*1000000)) == manhattan(p_i(n*1000000)).min())

# Part 1 - doesn't acceleration just win out?

particles `[ 21, 159, 285, 442, 457]` have the equal lowest acceleration.

particle 457 starts nearest the origin and moves slowest at the beginning (all of these particles are have a component of their velocity in the same direction as their respective accelerations, i.e. none is currently slowing)

therefore the part 1 answer is 457

In [8]:
# def cull(a, c=None):
#     '''If called without a second argument: remove duplicate elements from array, returning the new array and
#     a list of the indices of the elements kept from the original array
    
#     If called with the second element, cull the array based on the indices of the second argument'''
#     if c is None:
#         arr, c = np.unique(a, axis=0, return_counts=True)
#         out = arr[np.where(c == 1)]
#         mask = np.all(np.where(np.isin(a, out)),axis=0)
#         return out, mask
#     else:
#         out = a[c]
#         return out, c


In [9]:
test_data2 = '''p=<-6,0,0>, v=< 3,0,0>, a=< 0,0,0>    
p=<-4,0,0>, v=< 2,0,0>, a=< 0,0,0>
p=<-2,0,0>, v=< 1,0,0>, a=< 0,0,0>
p=< 3,0,0>, v=<-1,0,0>, a=< 0,0,0>'''.splitlines()

In [10]:
#p0, v0, a0 = parse_lines(test_data2)
p0, v0, a0 = parse_lines(real_data)

In [11]:
iters = 0
p, v, a = map(pd.DataFrame, [p0, v0, a0])
lastcount = len(p) + 1

In [12]:
while iters < 100:
    if iters % 10000 == 0:
        print(f'iteration {iters:8}')
    v = v + a
    p = p + v
    mask = ~p.duplicated(keep=False)
    v = v[mask].reset_index(drop=True)
    a = a[mask].reset_index(drop=True)
    p = p[mask].reset_index(drop=True)
    count = len(p)
    iters += 1
    if count != lastcount:
        print(f'iteration {iters:8}: count = {count}', flush=True)
        lastcount = count



iteration        0
iteration        1: count = 1000
iteration       10: count = 998
iteration       18: count = 996
iteration       35: count = 994
iteration       36: count = 992


In [13]:
len(p)

992

particles `[ 21, 159, 285, 442, 457]` have the equal lowest acceleration.

particle 457 starts nearest the origin and moves slowest at the beginning

therefore the part 1 answer is 457

# Pandas version

In [14]:
p0, v0, a0 = parse_lines(real_data)


sdf = pd.concat(map(pd.DataFrame, [p0, v0, a0]), axis=1, ignore_index=True)
V = ['v0', 'v1', 'v2']
A = ['a0', 'a1', 'a2']
P = ['p0', 'p1', 'p2']

sdf.columns = P + V + A


In [15]:
sdf.head()

,p0,p1,p2,v0,v1,v2,a0,a1,a2
0,-833,-499,1,84,17,1,-4,1,1
1,-168,3586,1,-61,-58,1,7,-13,8
2,364,223,7,31,-11,1,-5,0,3
3,769,-854,5,-20,4,4,0,1,9
4,6985,-3666,3,-112,99,3,-4,0,4


In [16]:

iters = 0
lastcount = len(sdf)+1
while iters < 100:
    sdf.loc[:, V] = sdf.loc[:, V] + sdf.loc[:, A].rename(columns=dict(zip(A, V)))
    sdf.loc[:, P] = sdf.loc[:, P] + sdf.loc[:, V].rename(columns=dict(zip(V, P)))
    
    mask = ~sdf.duplicated(subset=P, keep=False) # keep = False == don't include ANY dupes in output
    sdf = sdf[mask].reset_index(drop=True)
    count = len(sdf)
    iters += 1
    if count != lastcount:
        print(f'iteration {iters:8}: count = {count}', flush=True)
        lastcount = count

iteration        1: count = 1000
iteration       10: count = 998
iteration       18: count = 996
iteration       35: count = 994
iteration       36: count = 992


In [17]:
sdf.head()

,p0,p1,p2,v0,v1,v2,a0,a1,a2
0,-12633,6251,5151,-316,117,101,-4,1,1
1,29082,-67864,40501,639,-1358,801,7,-13,8
2,-21786,-877,15257,-469,-11,301,-5,0,3
3,-1231,4596,45855,-20,104,904,0,1,9
4,-24415,6234,20503,-512,99,403,-4,0,4


In [18]:
a= pd.DataFrame([[0], [0], [1], [2]])

In [19]:
a[~a.duplicated(keep=False)].reset_index(drop=True)

,0
0,1
1,2
